In [1]:
from mp_api.client import MPRester

from api_key import APIKEY

API_KEY = APIKEY

materials = ["mp-2657", "mp-390"]
bandstruct = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        try:
            band_structure = mpr.get_bandstructure_by_material_id(mp_id)
            bandstruct.append(band_structure)
        except Exception as e:
            print(f"Failed to retrieve band structure for {mp_id}: {e}")


Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
cbm_data = []
cbm_energy = []

vbm_data = []
vbm_energy = []

for band_structure in bandstruct:
    try:
        cbm = band_structure.get_cbm()
        cbm_data.append(cbm)

        vbm = band_structure.get_vbm()
        vbm_data.append(vbm)
       
        print("band_structures analyzed")
    except Exception as e:
        print(f"fail: {e}")

for cbm in cbm_data:
    try:
        cbmeng = cbm['energy']
        cbm_energy.append(cbmeng)
        print("CBM energy sucess")
    except Exception as e:
        print(f"fail: {e}")

for vbm in vbm_data:
    try:
        vbmeng = vbm['energy']
        vbm_energy.append(vbmeng)
        print("vbm energy sucess")
    except Exception as e:
        print(f"fail: {e}")

print(cbm_energy)
print(vbm_energy)

    


band_structures analyzed
band_structures analyzed
CBM energy sucess
CBM energy sucess
vbm energy sucess
vbm energy sucess
[4.9826, 3.9525]
[3.2016, 1.8939]
